In [38]:
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from simple_salesforce import Salesforce, format_soql, format_external_id
import paramiko
import datetime
from datetime import date
import re
from collections import OrderedDict
import simpler_sf
simpler_sf.simple_salesforce()
import simple_salesforce
from simple_salesforce.exceptions import SalesforceMalformedRequest

In [39]:
with open("login.json", "r") as j:
    misDatos = json.load(j)

In [40]:
#para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
#para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [41]:
fecha = datetime.date.today()
dia_hoy = str(fecha.day)
mes_hoy = str(fecha.month)
ano_hoy = str(fecha.year)
if len(mes_hoy) == 1:
    mes_hoy = "0" + mes_hoy
if len(dia_hoy) == 1:
    dia_hoy = "0" + dia_hoy
archivoDia = dia_hoy + mes_hoy + ano_hoy

nombreCarpetaHost = "/care_assistance/BCI_a_CAREASSIST/PRODUCCION" + "/"
nombreArchivoHost = "VentaCARE_ASSISTANCE_" + str(archivoDia) + ".xlsx"
#nombreArchivoHost = "VentaCARE_ASSISTANCE_" + '22112023' + ".xlsx"
descargaDeArchivo = nombreCarpetaHost + nombreArchivoHost

# Direccion descarga local
direccionDescargaLocal = (
    "F:/Procesos Automaticos/BCI Canal Individual/" + nombreArchivoHost
)

print(fecha)
print(archivoDia)
print(nombreArchivoHost)
print(descargaDeArchivo)

2024-02-23
23022024
VentaCARE_ASSISTANCE_23022024.xlsx
/care_assistance/BCI_a_CAREASSIST/PRODUCCION/VentaCARE_ASSISTANCE_23022024.xlsx


In [42]:
host = "sftp.bciseguros.cl"
puerto = 22
transporte = paramiko.Transport((host, puerto))
usuario = "pcerda_cas"
clave = "wB6tfCRc"
transporte.connect(None, usuario, clave)
sftp = paramiko.SFTPClient.from_transport(transporte)

Coneccion a carpeta sft y rescate de archivo


In [43]:
# Download
filepath = descargaDeArchivo
localpath = direccionDescargaLocal
sftp.get(filepath, localpath)

In [44]:
if sftp:
    sftp.close()
if transporte:
    transporte.close()

Inicio Manipulacion de Excel descargado


In [45]:
archivoBCI = pd.read_excel(direccionDescargaLocal)
archivoBCI.shape


c:\Users\jmartinez\.conda\envs\salesforce\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(14095, 17)

In [46]:
archivoBCI.dtypes


POLIZA                       int64
FECINIVIG           datetime64[ns]
FECFINVIG           datetime64[ns]
CODPROD                     object
CODPLAN                     object
DESCRIPCION_PLAN            object
RUT_TITULAR                  int64
DV_TITULAR                  object
RUT_ASEGURADO              float64
DV_ASEGURADO                object
SEXO                        object
PARENTESCO                  object
NOMBRE_ASEGURADO            object
FECHA_NACIMIENTO    datetime64[ns]
CORREO                      object
DIREC                       object
CELULAR                    float64
dtype: object

In [47]:
archivoBCI.RUT_ASEGURADO.fillna(archivoBCI["RUT_TITULAR"], inplace=True)
archivoBCI.DV_ASEGURADO.fillna(archivoBCI["DV_TITULAR"], inplace=True)
archivoBCI.SEXO.fillna("0", inplace=True)
archivoBCI.CORREO.fillna("", inplace=True)
archivoBCI.CELULAR.fillna("0", inplace=True)
archivoBCI.DIREC.fillna("", inplace=True)
archivoBCI.NOMBRE_ASEGURADO.fillna("NN NN", inplace=True)
archivoBCI["RUT_ASEGURADO"] = archivoBCI["RUT_ASEGURADO"].apply(np.int64)
archivoBCI["CELULAR"] = archivoBCI["CELULAR"].astype(int)
archivoBCI["CELULAR"] = archivoBCI["CELULAR"].apply(np.int64)


archivoBCI.head(-50)

,POLIZA,FECINIVIG,FECFINVIG,CODPROD,CODPLAN,DESCRIPCION_PLAN,RUT_TITULAR,DV_TITULAR,RUT_ASEGURADO,DV_ASEGURADO,SEXO,PARENTESCO,NOMBRE_ASEGURADO,FECHA_NACIMIENTO,CORREO,DIREC,CELULAR
0,2597777,2023-04-25,2024-04-24,SAIN,M9,SALUD INDIVIDUAL PLAN 80 RECOVER,100,7,100,7,MASCULINO,TITULAR MASCULINO,RUT DE PRUEBA CERTIFICACION,1991-01-01,PAULITALORENA@HOTMAIL.COM,PASAJE MANCO CAPAC NRO 1745,0
1,2597778,2023-05-03,2024-05-02,SAIN,M7,SALUD INDIVIDUAL PLAN 60 RECOVER,100,7,100,7,MASCULINO,TITULAR MASCULINO,RUT DE PRUEBA CERTIFICACION,1991-01-01,PAULITALORENA@HOTMAIL.COM,PASAJE MANCO CAPAC NRO 1745,0
2,2597779,2023-05-03,2024-05-02,SAIN,M8,SALUD INDIVIDUAL PLAN 70 RECOVER,100,7,100,7,MASCULINO,TITULAR MASCULINO,RUT DE PRUEBA CERTIFICACION,1991-01-01,PAULITALORENA@HOTMAIL.COM,PASAJE MANCO CAPAC NRO 1745,0
3,2601791,2023-06-01,2024-05-31,SAIN,M9,SALUD INDIVIDUAL PLAN 80 RECOVER,15583606,7,15583606,7,MASCULINO,TITULAR MASCULINO,JESUS NOE REY ACEVEDO,1983-02-03,contaservicoser@gmail.com,SANTIAGO,933111465
4,2601791,2023-06-01,2024-05-31,SAIN,M9,SALUD INDIVIDUAL PLAN 80 RECOVER,15583606,7,15797990,6,FEMENINO,ESPOSA,Gajardo Pradenas Cynthia Del Pilar,1984-04-02,cgajardo34@gmail.com,SANTIAGO,996766488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14040,2664512,2023-11-02,2024-11-01,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,16370704,7,16370704,7,MASCULINO,TITULAR MASCULINO,FRANCISCO JAVIER ARANGUIZ GUTIERREZ,1987-10-07,FJARANGUIZ@GMAIL.COM,NO INFORMADO,997761126
14041,2664516,2023-11-02,2024-11-01,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,17622827,K,17622827,K,FEMENINO,TITULAR FEMENINO,NINOSKA VICTORIA MARTINEZ CARO,1990-10-24,NINOSKAVICTORIAA@GMAIL.COM,SANTIAGO,956186074
14042,2664851,2023-11-03,2024-11-02,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,16369391,7,16369391,7,MASCULINO,TITULAR MASCULINO,PABLO RODRIGO SANHUEZA CASTILLO,1987-08-10,elsangus@gmail.com,CALLE NUEVA 5331,987319468
14043,2664851,2023-11-03,2024-11-02,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,16369391,7,16015099,8,FEMENINO,ESPOSA,NATALIA ANDREA HURTADO ACUNA,1985-01-11,NATI.HURTADO@GMAIL.COM,SANTIAGO,984496090


In [48]:
def isValid(email):
    if re.fullmatch(regex, email):
        return email
    else:
        return ""


def nombreFuncion(nombreCompleto):
    if len(nombreCompleto) >= 2:
        nombresJuntos = nombreCompleto[0] + " " + nombreCompleto[1]
        return nombresJuntos
    elif len(nombreCompleto) == 1:
        nombresJuntos = nombreCompleto[0]
        return nombresJuntos
    else:
        nombresJuntos = "NN NN"
        return nombresJuntos
    print(len(nombresJuntos) + nombresJuntos)


def apellidoFuncion(nombreCompleto):
    if len(nombreCompleto) >= 2:
        nombresJuntos = nombreCompleto[-2] + " " + nombreCompleto[-1]
        return nombresJuntos
    elif len(nombreCompleto) == 1:
        nombresJuntos = nombreCompleto[-1]
        return nombresJuntos
    else:
        nombresJuntos = "NN NN"
        return nombresJuntos
    print(len(nombresJuntos) + nombresJuntos)


""" def apellidoFuncion(nombreCompleto):
  nombreCompleto = str(nombreCompleto)
  apellidos = re.split('\W+', nombreCompleto)
  if len(apellidos)==1:
    return apellidos[0]
  elif len(apellidos)==2:
    nombresJuntos = apellidos[0] + " " + apellidos[1]
    return nombresJuntos
    
  elif len(apellidos)==3:
    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]
    archivoBCI["apellido"] = apellidos[2]
  else:
    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]
    archivoBCI["apellido"] = apellidos[2] + " " + apellidos[3] """

# archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]
# archivoBCI["apellido"] = apellidos[2] + " " + apellidos[3]
# print (apellidos)

' def apellidoFuncion(nombreCompleto):\n  nombreCompleto = str(nombreCompleto)\n  apellidos = re.split(\'\\W+\', nombreCompleto)\n  if len(apellidos)==1:\n    return apellidos[0]\n  elif len(apellidos)==2:\n    nombresJuntos = apellidos[0] + " " + apellidos[1]\n    return nombresJuntos\n    \n  elif len(apellidos)==3:\n    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]\n    archivoBCI["apellido"] = apellidos[2]\n  else:\n    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]\n    archivoBCI["apellido"] = apellidos[2] + " " + apellidos[3] '

In [49]:
archivoBCI["Rut Titular"] = archivoBCI["RUT_TITULAR"].astype(str) + archivoBCI[
    "DV_TITULAR"
].astype(str)
archivoBCI["Rut Titular"] = archivoBCI["Rut Titular"].str.replace(" ", "").str.upper()

archivoBCI["Rut Carga"] = archivoBCI["RUT_ASEGURADO"].astype(str) + archivoBCI[
    "DV_ASEGURADO"
].astype(str)
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace(" ", "").str.upper()

archivoBCI["poliza"] = archivoBCI["POLIZA"].astype(str)

general_Sexo = {
    "M": "Hombre",
    "m": "Hombre",
    "Masculino": "Hombre",
    "Femenino": "Mujer",
    "": "Prefiero no responder",
    "HOMBRE": "Hombre",
    "MUJER": "Mujer",
    "F": "Mujer",
    "O": "O",
    "Ambos": "Prefiero no responder",
    "N": "Prefiero no responder",
    "Otro": "Prefiero no responder",
    "n": "Prefiero no responder",
    "N": "Prefiero no responder",
    "FEMENINO": "Mujer",
    "f": "Mujer",
    "MASCULINO": "Hombre",
    "m": "Hombre",
    "M ": "Hombre",
    "F ": "Mujer",
    "Varón": "Hombre",
    "0": "",
}
archivoBCI.replace({"SEXO": general_Sexo}, inplace=True)
archivoBCI["sexo"] = archivoBCI["SEXO"].astype(str).str.title()

general_email = {
    "no informado": "",
}
archivoBCI["email"] = archivoBCI["CORREO"].astype(str).str.lower()
archivoBCI["email"] = archivoBCI["email"].apply(lambda y: isValid(y))
archivoBCI.replace({"email": general_email}, inplace=True)

general_direccion = {
    "no informado": "",
    "SN": "",
    "sn": "",
    "s/d": "",
    "1 1": "",
    "0 0": "",
    "sn 0": "",
}
archivoBCI["direccion"] = archivoBCI["DIREC"].astype(str).str.lower()
archivoBCI.replace({"direccion": general_direccion}, inplace=True)
archivoBCI["direccion"] = archivoBCI["direccion"].str.title()

general_Celular = {
    0: "",
}
archivoBCI.replace({"CELULAR": general_Celular}, inplace=True)
archivoBCI["celular"] = archivoBCI["CELULAR"].astype(str).str.lower()
archivoBCI["celular"] = archivoBCI["celular"].str.replace("NaN", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("Nan", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("-", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("No Informado", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("_", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("+", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("nan", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace(" ", "")


archivoBCI["nombres"] = archivoBCI["NOMBRE_ASEGURADO"].str.split(n=2)
archivoBCI["apellidos"] = archivoBCI["NOMBRE_ASEGURADO"].str.rsplit(n=2)
archivoBCI["nombres"] = (
    archivoBCI["nombres"].apply(lambda y: nombreFuncion(y)).str.title()
)
archivoBCI["apellidos"] = (
    archivoBCI["apellidos"].apply(lambda y: apellidoFuncion(y)).str.title()
)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_24884\2221294489.py:70: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  archivoBCI["celular"] = archivoBCI["celular"].str.replace("+", "")


In [50]:
archivoBCI.head(-50)


,POLIZA,FECINIVIG,FECFINVIG,CODPROD,CODPLAN,DESCRIPCION_PLAN,RUT_TITULAR,DV_TITULAR,RUT_ASEGURADO,DV_ASEGURADO,...,CELULAR,Rut Titular,Rut Carga,poliza,sexo,email,direccion,celular,nombres,apellidos
0,2597777,2023-04-25,2024-04-24,SAIN,M9,SALUD INDIVIDUAL PLAN 80 RECOVER,100,7,100,7,...,,1007,1007,2597777,Hombre,paulitalorena@hotmail.com,Pasaje Manco Capac Nro 1745,,Rut De,Prueba Certificacion
1,2597778,2023-05-03,2024-05-02,SAIN,M7,SALUD INDIVIDUAL PLAN 60 RECOVER,100,7,100,7,...,,1007,1007,2597778,Hombre,paulitalorena@hotmail.com,Pasaje Manco Capac Nro 1745,,Rut De,Prueba Certificacion
2,2597779,2023-05-03,2024-05-02,SAIN,M8,SALUD INDIVIDUAL PLAN 70 RECOVER,100,7,100,7,...,,1007,1007,2597779,Hombre,paulitalorena@hotmail.com,Pasaje Manco Capac Nro 1745,,Rut De,Prueba Certificacion
3,2601791,2023-06-01,2024-05-31,SAIN,M9,SALUD INDIVIDUAL PLAN 80 RECOVER,15583606,7,15583606,7,...,933111465,155836067,155836067,2601791,Hombre,contaservicoser@gmail.com,Santiago,933111465,Jesus Noe,Rey Acevedo
4,2601791,2023-06-01,2024-05-31,SAIN,M9,SALUD INDIVIDUAL PLAN 80 RECOVER,15583606,7,15797990,6,...,996766488,155836067,157979906,2601791,Mujer,cgajardo34@gmail.com,Santiago,996766488,Gajardo Pradenas,Del Pilar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14040,2664512,2023-11-02,2024-11-01,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,16370704,7,16370704,7,...,997761126,163707047,163707047,2664512,Hombre,fjaranguiz@gmail.com,,997761126,Francisco Javier,Aranguiz Gutierrez
14041,2664516,2023-11-02,2024-11-01,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,17622827,K,17622827,K,...,956186074,17622827K,17622827K,2664516,Mujer,ninoskavictoriaa@gmail.com,Santiago,956186074,Ninoska Victoria,Martinez Caro
14042,2664851,2023-11-03,2024-11-02,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,16369391,7,16369391,7,...,987319468,163693917,163693917,2664851,Hombre,elsangus@gmail.com,Calle Nueva 5331,987319468,Pablo Rodrigo,Sanhueza Castillo
14043,2664851,2023-11-03,2024-11-02,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,16369391,7,16015099,8,...,984496090,163693917,160150998,2664851,Mujer,nati.hurtado@gmail.com,Santiago,984496090,Natalia Andrea,Hurtado Acuna


In [51]:
# archivoBCI = archivoBCI[archivoBCI["RUT_ASEGURADO"]== "0"]
# archivoBCI.to_excel("F:/Procesos Automaticos/BCI Canal Individual/derty.xlsx",index=False)

In [52]:
def revisaDict(ver):
    numbers = ver
    for key, value in numbers.items():
        print(key, "->", value)
        if key == "IdentificationId__pc":
            return value


""" def revisaDict(ver):
    numbers = ver
    for key, value in numbers.items():
        print(key, "->", value)
        if key == "IdentificationId__pc":
            return value """


' def revisaDict(ver):\n    numbers = ver\n    for key, value in numbers.items():\n        print(key, "->", value)\n        if key == "IdentificationId__pc":\n            return value '

In [53]:
""" results = sf.query_all(
    
    select Paciente__r.IdentificationId__pc from Paciente_Programas__c where padre__c = 'BCI Seguros Individuales'
  
)
pacienteProgr = pd.DataFrame(results["records"])
# Pacientes_SF["nuevo"] = Paciente__r.OrderedDict.IdentificationId__pc
# String acctName = carol.Account.Name;

pacienteProgr["sfRut"] = pacienteProgr["Paciente__r"].apply(lambda y: revisaDict(y))

pacienteProgr["sfRut"] = pacienteProgr["sfRut"].astype(str).str.upper()

pacienteProgr = pacienteProgr[["sfRut",]]


pacienteProgr.shape """


' results = sf.query_all(\n    \n    select Paciente__r.IdentificationId__pc from Paciente_Programas__c where padre__c = \'BCI Seguros Individuales\'\n  \n)\npacienteProgr = pd.DataFrame(results["records"])\n# Pacientes_SF["nuevo"] = Paciente__r.OrderedDict.IdentificationId__pc\n# String acctName = carol.Account.Name;\n\npacienteProgr["sfRut"] = pacienteProgr["Paciente__r"].apply(lambda y: revisaDict(y))\n\npacienteProgr["sfRut"] = pacienteProgr["sfRut"].astype(str).str.upper()\n\npacienteProgr = pacienteProgr[["sfRut",]]\n\n\npacienteProgr.shape '

In [54]:
#pacienteProgr.head()

In [55]:

campaña_BCI = "BCI Seguros Individuales"
query_pacientePrograma = (
    "select Id, Campa_a__c, Paciente__r.IdentificationId__pc, N_Documento_Titular__c,Paciente__r.FirstName, Paciente__r.LastName,Empresa__r.P_liza__c, Empresa__r.Rut_Empresa__c, Empresa__r.Identificador__c,Convenio_Activo__c,Relaci_n_Con_Asegurado__c from EmpresaContactoConvenios__c where Empresa__r.Campa_a__c = '" + campaña_BCI +"'"
)
PAcientes_q_ya_estan0 = sf.smart_query(query_pacientePrograma)

campaña_BCI = "BCI Seguros Colectivos"
query_pacientePrograma1 = (
    "select Id, Campa_a__c, Paciente__r.IdentificationId__pc, N_Documento_Titular__c,Paciente__r.FirstName, Paciente__r.LastName,Empresa__r.P_liza__c, Empresa__r.Rut_Empresa__c, Empresa__r.Identificador__c,Convenio_Activo__c,Relaci_n_Con_Asegurado__c from EmpresaContactoConvenios__c where Empresa__r.Campa_a__c = '" + campaña_BCI +"'"
)
PAcientes_q_ya_estan1 = sf.smart_query(query_pacientePrograma1)

campaña_BCI = "BCI Seguros Colaboradores"
query_pacientePrograma2 = (
    "select Id, Campa_a__c, Paciente__r.IdentificationId__pc, N_Documento_Titular__c,Paciente__r.FirstName, Paciente__r.LastName,Empresa__r.P_liza__c, Empresa__r.Rut_Empresa__c, Empresa__r.Identificador__c,Convenio_Activo__c,Relaci_n_Con_Asegurado__c from EmpresaContactoConvenios__c where Empresa__r.Campa_a__c = '" + campaña_BCI +"'"
)
PAcientes_q_ya_estan2 = sf.smart_query(query_pacientePrograma2)



PAcientes_q_ya_estan = pd.concat(
    [
        PAcientes_q_ya_estan0,
        PAcientes_q_ya_estan1,
        PAcientes_q_ya_estan2,
        
    ],
    axis=0,
)

PAcientes_q_ya_estan["sfRut"] = PAcientes_q_ya_estan["Paciente__r.IdentificationId__pc"]
PAcientes_q_ya_estan["Rut"] = PAcientes_q_ya_estan["Paciente__r.IdentificationId__pc"]
PAcientes_q_ya_estan.head()

PAcientes_q_ya_estan.to_excel(
    "F:/Empresas SF/reportes/rutsBci.xlsx", index=False
)


PAcientes_q_ya_estan.shape




100%|██████████| 1/1 [00:10<00:00, 10.64s/it]


(36821, 13)

In [56]:
PAcientes_q_ya_estan["sfRut"] = PAcientes_q_ya_estan["sfRut"].astype(str).str.upper()
PAcientes_q_ya_estan["sfRut"] = PAcientes_q_ya_estan["sfRut"].str.replace(" ", "")
PAcientes_q_ya_estan["sfRut"] = PAcientes_q_ya_estan["sfRut"].str.replace("-", "")

archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].astype(str).str.upper()
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace(" ", "")
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace("-", "")

archivoBCI["RutEstaoNo"] = archivoBCI["Rut Carga"].isin(PAcientes_q_ya_estan["sfRut"])

archivoBCI = archivoBCI[archivoBCI["RutEstaoNo"] == False]


archivoBCI.drop(
    archivoBCI[
        (archivoBCI["nombres"] == "Nn Nn") & (archivoBCI["apellidos"] == "Nn Nn")
    ].index,
    inplace=True,
)
archivoBCI = archivoBCI.drop_duplicates(["Rut Carga"], keep="last")

archivoBCI = archivoBCI[
    ["Rut Carga", "sexo", "direccion", "email", "celular", "nombres", "apellidos","Rut Titular"]
]

archivoBCI = archivoBCI.rename(
    columns={
        "Rut Carga": "IdentificationId__pc",
        "sexo": "HealthCloudGA__Gender__pc",
        "direccion": "BillingStreet",
        "email": "PersonEmail",
        "celular": "Phone",
        "nombres": "FirstName",
        "apellidos": "LastName",
    }
)
archivoBCI["BillingCountry"] = "Chile"
archivoBCI["Identification_Type__c"] = "RUT"
archivoBCI["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
archivoBCI["RecordTypeId"] = "0128c000002Byp4AAC"
archivoBCI["Cuenta_validada__c"] = "True"
archivoBCI["Validaci_n__c"] = "Validada"
archivoBCI["Email_Seteado__c"] = np.where(
    archivoBCI["PersonEmail"] != "", "True", "False"
)
archivoBCI["Cuenta_validada__c"] = "True"
archivoBCI.to_excel(
    "F:/Procesos Automaticos/BCI Canal Individual/derty.xlsx", index=False
)


archivoBCI.head()


,IdentificationId__pc,HealthCloudGA__Gender__pc,BillingStreet,PersonEmail,Phone,FirstName,LastName,Rut Titular,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c,RecordTypeId,Cuenta_validada__c,Validaci_n__c,Email_Seteado__c
11941,1007,Hombre,Pasaje Manco Capac Nro 1745,paulitalorena@hotmail.com,,Rut De,Prueba Certificacion,1007,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True
13717,81465878,Hombre,Santiago Sn,mimdg@hotmail.com,985318612,Manuel Ignacio,Morales Diaz,81465878,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True
13718,172674186,Hombre,Los Duendes 7445 Maipu,ahumadalavinma@gmail.com,951330401,Matias Ignacio,Ahumada Lavin,172674186,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True
13719,115171224,Mujer,Santiago,lcontreras@indumac.cl,976192501,Lorens Paola,Contreras Herrera,115171224,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True
13721,136352997,Mujer,Santiago,susanagonzaleztorres@gmail.com,989438567,Susana Patricia,Gonzalez Torres,136352997,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True


In [57]:
""" derty = sf.quick_search("153298939")
#print(derty)

revisaDict(derty)
 """
""" soql_query =  select Id, Paciente__r.IdentificationId__pc from Paciente_Programas__c where padre__c = 'BCI Seguros Individuales' 
soql_result = sf.query_all(soql_query)
df = pd.DataFrame(soql_result.get("records"))
df = df.drop(columns=["attributes"])
#df = df.drop(columns="Id")
list_of_records = df.to_dict("index") """

""" derty = sf.quick_search("153298939")
print(derty) """

""" for listado in list_of_records:
    list_of_records[listado]["Paciente__r"]["IdentificationId__pc"]
    juanka =  len(list_of_records)
    #print(list_of_records[listado]["Paciente__r"]["IdentificationId__pc"])
    print(list_of_records[listado]["Id"]) 
    
list_of_records[1]"""

' for listado in list_of_records:\n    list_of_records[listado]["Paciente__r"]["IdentificationId__pc"]\n    juanka =  len(list_of_records)\n    #print(list_of_records[listado]["Paciente__r"]["IdentificationId__pc"])\n    print(list_of_records[listado]["Id"]) \n    \nlist_of_records[1]'

Weeeeeeeeeeeenooo


In [58]:
# rut = "124942012"
def traerId(rut):
    if (len(rut)>=8):
        busquedaDatos = "FIND {" + rut + "} RETURNING Account(Id,IdentificationId__pc WHERE IdentificationId__pc = '" + rut + "')"
        derty = dict(sf.search(busquedaDatos))
        if derty["searchRecords"] != []:
            if derty["searchRecords"][0]["attributes"]["type"] == "Account":
                jote = derty["searchRecords"][0]["Id"]
                dertyd = derty["searchRecords"][0]["IdentificationId__pc"]
                print(jote + " " + dertyd)
                return jote
            else:
                print("no")
        else:
            print("vacio")
    else:
        return ""      
    
def cambiaranone(Id2):
    if Id2 is not None and len(Id2) >= 7:
        return Id2
    else:
        return "NoNe"       

In [59]:
archivoBCI.shape

(28, 15)

In [60]:

archivoBCI["Paciente__c"] = archivoBCI["IdentificationId__pc"].apply(lambda y: traerId(y))




archivoBCI["Empresa__c"] = "a3yRN0000006rMRYAY"
archivoBCI["N_Documento_Titular__c"] = archivoBCI["Rut Titular"]

archivoBCI["esta"] = archivoBCI["Paciente__c"].apply(lambda y: cambiaranone(y))
archivoBCI = archivoBCI[archivoBCI["esta"] != "NoNe" ]


#Empresa__c, Paciente__c, N_Documento_Titular__c,
archivoBCI.head()

vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
0018c00002bKdsTAAS 122207080
vacio
vacio
vacio
vacio
0018c00002eW2tqAAC 14103460K
vacio
vacio
vacio
vacio
vacio


,IdentificationId__pc,HealthCloudGA__Gender__pc,BillingStreet,PersonEmail,Phone,FirstName,LastName,Rut Titular,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c,RecordTypeId,Cuenta_validada__c,Validaci_n__c,Email_Seteado__c,Paciente__c,Empresa__c,N_Documento_Titular__c,esta
13838,122207080,Mujer,Santiago,carla.recabarren50@gmail.com,994205453,Recabarren Arancibia,Carla Soledad,122207080,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,0018c00002bKdsTAAS,a3yRN0000006rMRYAY,122207080,0018c00002bKdsTAAS
13843,14103460K,Hombre,Santiago Sn,rmunoz@uc.cl,996993687,Roberto Pablo,Muñoz Soria,14103460K,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,0018c00002eW2tqAAC,a3yRN0000006rMRYAY,14103460K,0018c00002eW2tqAAC


In [61]:
#archivoBCI = archivoBCI["Paciente__c"] Paciente__c

In [62]:
archivoBCI = archivoBCI[archivoBCI["esta"] != "NoNe" ]
archivoBCI.head()


,IdentificationId__pc,HealthCloudGA__Gender__pc,BillingStreet,PersonEmail,Phone,FirstName,LastName,Rut Titular,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c,RecordTypeId,Cuenta_validada__c,Validaci_n__c,Email_Seteado__c,Paciente__c,Empresa__c,N_Documento_Titular__c,esta
13838,122207080,Mujer,Santiago,carla.recabarren50@gmail.com,994205453,Recabarren Arancibia,Carla Soledad,122207080,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,0018c00002bKdsTAAS,a3yRN0000006rMRYAY,122207080,0018c00002bKdsTAAS
13843,14103460K,Hombre,Santiago Sn,rmunoz@uc.cl,996993687,Roberto Pablo,Muñoz Soria,14103460K,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,0018c00002eW2tqAAC,a3yRN0000006rMRYAY,14103460K,0018c00002eW2tqAAC


In [63]:
archivoDeCarga = (
    "F:/Procesos Automaticos/BCI Canal Individual/" + str(archivoDia) + ".xlsx"
)

archivoBCI.to_excel(archivoDeCarga, index=False)

archivoBCI.head()

,IdentificationId__pc,HealthCloudGA__Gender__pc,BillingStreet,PersonEmail,Phone,FirstName,LastName,Rut Titular,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c,RecordTypeId,Cuenta_validada__c,Validaci_n__c,Email_Seteado__c,Paciente__c,Empresa__c,N_Documento_Titular__c,esta
13838,122207080,Mujer,Santiago,carla.recabarren50@gmail.com,994205453,Recabarren Arancibia,Carla Soledad,122207080,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,0018c00002bKdsTAAS,a3yRN0000006rMRYAY,122207080,0018c00002bKdsTAAS
13843,14103460K,Hombre,Santiago Sn,rmunoz@uc.cl,996993687,Roberto Pablo,Muñoz Soria,14103460K,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,0018c00002eW2tqAAC,a3yRN0000006rMRYAY,14103460K,0018c00002eW2tqAAC


In [64]:
def traerId(rut):
    if len(rut) >= 8:
        busquedaDatos = "FIND {" + rut + "} RETURNING Account(Id,IdentificationId__pc)"
        derty = dict(sf.search(busquedaDatos))
        if derty["searchRecords"] != []:
            if derty["searchRecords"][0]["attributes"]["type"] == "Account":
                jote = derty["searchRecords"][0]["Id"]
                dertyd = derty["searchRecords"][0]["IdentificationId__pc"]
                print(jote + " " + dertyd)
                return jote
            else:
                print("no")
        else:
            print("vacio")
    else:
        return "None"


def traerContact(ide):
    ide = str(ide)
    if len(ide) >= 12:
        idcontact = sf.Account.get(ide)
        return idcontact["PersonContactId"]
    else:
        return "NONES"


def agregarPaciente(listado):
    vListado = listado.split(",")
    
    derty = None
    #vListado[9] = datetime.strptime(vListado[9], "%Y-%m-%d %H:%M:%S")
        
        
    if (vListado[9] == ""):                               
        try:
            derty = dict(
                sf.Account.create(
                    # derty = sf.Account.create(
                    {
                        "FirstName": vListado[0],
                        "LastName": vListado[1],
                        "IdentificationId__pc": vListado[2],
                        "RecordTypeId": vListado[3],
                        "BillingCountry": vListado[4],
                        "Identification_Type__c": vListado[5],
                        "Pa_s_del_Paciente__c": vListado[6],
                        "Cuenta_validada__c": True,
                        "PersonEmail": vListado[8],  
                                          
                        "HealthCloudGA__Gender__pc": vListado[10],
                        "Phone": vListado[11],
                        "BillingCity": vListado[12],
                        "BillingState": vListado[13],
                        "BillingStreet": vListado[14],
                        "Email_Seteado__c": bool(vListado[15]),
                    }
                )
            )
            # return derty["id"]
        except SalesforceMalformedRequest as e:
            print(f"Malformed request error: {e}")
            error = {e}

        if derty is not None:
            return derty["id"]
        else:
            traerId(vListado[2])
    else:
        try:
            derty = dict(
                sf.Account.create(
                    # derty = sf.Account.create(
                    {
                        "FirstName": vListado[0],
                        "LastName": vListado[1],
                        "IdentificationId__pc": vListado[2],
                        "RecordTypeId": vListado[3],
                        "BillingCountry": vListado[4],
                        "Identification_Type__c": vListado[5],
                        "Pa_s_del_Paciente__c": vListado[6],
                        "Cuenta_validada__c": True,
                        "PersonEmail": vListado[8],
                        "PersonBirthdate": vListado[9] ,
                        #"PersonBirthdate":datetime.strptime(vListado[9], "%Y-%m-%d %H:%M:%S").formatGMT('yyyy-MM-dd\'T\'HH:mm:ss.SSS\'Z\'' + '"'),
                        "HealthCloudGA__Gender__pc": vListado[10],
                        "Phone": vListado[11],
                        "BillingCity": vListado[12],
                        "BillingState": vListado[13],
                        "BillingStreet": vListado[14],
                        "Email_Seteado__c": bool(vListado[15]),
                    }
                )
            )
            # return derty["id"]
        except SalesforceMalformedRequest as e:
            print(f"Malformed request error: {e}")
            error = {e}

        if derty is not None:
            return derty["id"]
        else:
            traerId(vListado[2])
    # Verify that the Contact was created successfully
    # created_contact = sf.Contact.get(sf_res['id'])
    # print(created_contact)


""" sf_res =None
try:
    # Create the new Contact record in Salesforce
    sf_res = sf.Contact.create(new_contact)
except SalesforceMalformedRequest as e:
    if 'duplicateResut' in e[0]:
        print(f"Malformed request error: {e}")
        # Handle the error here as needed
        #Bypass SF Email duplicate rule by removing the email to create a SF Lead
        del new_contact['Email']
        sf_res = sf.Contact.create(new_contact)


if sf_res is not None:
    # Verify that the Contact was created successfully
    created_contact = sf.Contact.get(sf_res['id'])
    print(created_contact) """

' sf_res =None\ntry:\n    # Create the new Contact record in Salesforce\n    sf_res = sf.Contact.create(new_contact)\nexcept SalesforceMalformedRequest as e:\n    if \'duplicateResut\' in e[0]:\n        print(f"Malformed request error: {e}")\n        # Handle the error here as needed\n        #Bypass SF Email duplicate rule by removing the email to create a SF Lead\n        del new_contact[\'Email\']\n        sf_res = sf.Contact.create(new_contact)\n\n\nif sf_res is not None:\n    # Verify that the Contact was created successfully\n    created_contact = sf.Contact.get(sf_res[\'id\'])\n    print(created_contact) '

In [65]:

Subir_pacientes = archivoBCI
Subir_pacientes["dato_account"] = ""
if ("FirstName" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + Subir_pacientes[
        "FirstName"
    ].astype(str)
if ("LastName" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"] + "," + Subir_pacientes["LastName"].astype(str)
    )
if ("IdentificationId__pc" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["IdentificationId__pc"].astype(str)
    )
if ("RecordTypeId" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["RecordTypeId"].astype(str)
    )
if ("BillingCountry" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingCountry"].astype(str)
    )
if ("Identification_Type__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Identification_Type__c"].astype(str)
    )
if ("Pa_s_del_Paciente__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Pa_s_del_Paciente__c"].astype(str)
    )
if ("Cuenta_validada__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Cuenta_validada__c"].astype(str)
    )

if ("PersonEmail" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["PersonEmail"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","

if ("PersonBirthdate" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (Subir_pacientes["dato_account"] + "," + Subir_pacientes["PersonBirthdate"].apply(lambda x: x.isoformat() + "Z"))
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("HealthCloudGA__Gender__pc" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["HealthCloudGA__Gender__pc"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("Phone" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"] + "," + Subir_pacientes["Phone"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("BillingCity" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingCity"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("BillingState" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingState"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("BillingStreet" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingStreet"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("PersonEmail" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Email_Seteado__c"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","

# Subir_pacientes["dato_account"] =  Subir_pacientes["dato_account"] =

Subir_pacientes["Id_creado"] = Subir_pacientes["dato_account"].apply(
    lambda x: agregarPaciente(x)
)
""" Subir_pacientes["Contact_creado"] = Subir_pacientes["dato_account"].apply(
    lambda x: agregarPaciente(x)
) """

Malformed request error: Malformed request https://careassistance.my.salesforce.com/services/data/v42.0/sobjects/Account/. Response content: [{'message': 'se encontró un valor duplicado: IdentificationId__c duplica el valor en el registro con Id.: 0038c000035w1XBAAY', 'errorCode': 'DUPLICATE_VALUE', 'fields': []}]
0018c00002bKdsTAAS 122207080
Malformed request error: Malformed request https://careassistance.my.salesforce.com/services/data/v42.0/sobjects/Account/. Response content: [{'message': 'se encontró un valor duplicado: IdentificationId__c duplica el valor en el registro con Id.: 0038c000039EPP6AAO', 'errorCode': 'DUPLICATE_VALUE', 'fields': []}]
0018c00002eW2tqAAC 14103460K


' Subir_pacientes["Contact_creado"] = Subir_pacientes["dato_account"].apply(\n    lambda x: agregarPaciente(x)\n) '

In [66]:
PAcientes_q_ya_estan.head()
#PAcientes_q_ya_estan.shape


,Id,Campa_a__c,Paciente__r.IdentificationId__pc,Paciente__r.FirstName,Paciente__r.LastName,N_Documento_Titular__c,Empresa__r.P_liza__c,Empresa__r.Rut_Empresa__c,Empresa__r.Identificador__c,Convenio_Activo__c,Relaci_n_Con_Asegurado__c,sfRut,Rut
0,a40RN000001B4eAYAS,BCI Seguros Individuales,173090250,Diego,Daniel Uribe Oyanedel,173090250,-,-,BCI Seguros,True,Titular,173090250,173090250
1,a40RN000001B4eBYAS,BCI Seguros Individuales,173119216,Matías,Emir Uribe Obregón,173119216,-,-,BCI Seguros,True,Titular,173119216,173119216
2,a40RN000001B4eCYAS,BCI Seguros Individuales,173141211,Sebastian,Elias Carrera Astorga,173141211,-,-,BCI Seguros,True,Titular,173141211,173141211
3,a40RN000001B4eDYAS,BCI Seguros Individuales,173144962,Paloma Lopez,Lopez Lagos,173144962,-,-,BCI Seguros,True,Titular,173144962,173144962
4,a40RN000001B4eEYAS,BCI Seguros Individuales,173173814,Franco Andres,Alfaro Duijvesteijn,173173814,-,-,BCI Seguros,True,Titular,173173814,173173814


In [67]:
rut_yidEmCoCo = pd.DataFrame()
rut_yidEmCoCo["Id empreconcon"] = PAcientes_q_ya_estan["Id"]
rut_yidEmCoCo["rut"] =  PAcientes_q_ya_estan["Paciente__r.IdentificationId__pc"]

rut_yidEmCoCo.shape

(36821, 2)

In [68]:
rut_yidEmCoCo = rut_yidEmCoCo.drop_duplicates(["rut"], keep="last")
rut_yidEmCoCo.head()



,Id empreconcon,rut
0,a40RN000001B4eAYAS,173090250
1,a40RN000001B4eBYAS,173119216
2,a40RN000001B4eCYAS,173141211
3,a40RN000001B4eDYAS,173144962
4,a40RN000001B4eEYAS,173173814


In [69]:
rut_yidEmCoCo.shape

(36788, 2)

In [70]:
PAcientes_q_ya_estan["RutEstaoNo"] = PAcientes_q_ya_estan["Id"].isin(rut_yidEmCoCo["Id empreconcon"])
PAcientes_q_ya_estan.shape


(36821, 14)

In [71]:
noestan = PAcientes_q_ya_estan[(PAcientes_q_ya_estan["RutEstaoNo"] == False) & (PAcientes_q_ya_estan["Relaci_n_Con_Asegurado__c"] != "Titular")]
noestan.shape

(0, 14)

In [72]:
archivoDeCarga = (
    "F:/Procesos Automaticos/BCI Canal Individual/repetidos.xlsx"
)

noestan.to_excel(archivoDeCarga, index=False)

In [73]:
""" sf.Account.create(
    {
        "IdentificationId__pc": "12345678910",
        "HealthCloudGA__Gender__pc": "Mujer",
        "FirstName": "johan",
        "LastName": "Smith",
        "PersonEmail": "example@example.com",
        "BillingCountry": "Chile",
        "Identification_Type__c": "RUT",
        "Pa_s_del_Paciente__c": "a1U8c000005dAtnEAE",
        "RecordTypeId": "0128c000002Byp4AAC",
    }
)
 """

# sf.Account.update(1, {'Name': 'Test Account', ContactId: contact_id}) # doctest: +SKIP

' sf.Account.create(\n    {\n        "IdentificationId__pc": "12345678910",\n        "HealthCloudGA__Gender__pc": "Mujer",\n        "FirstName": "johan",\n        "LastName": "Smith",\n        "PersonEmail": "example@example.com",\n        "BillingCountry": "Chile",\n        "Identification_Type__c": "RUT",\n        "Pa_s_del_Paciente__c": "a1U8c000005dAtnEAE",\n        "RecordTypeId": "0128c000002Byp4AAC",\n    }\n)\n '

Codigo que dio github Copilot para la solucion del error de 0 en envio de correos

In [74]:
import win32com.client as win32
import os
import shutil

nombre_reporte = "Vida Security - Ferronor"
nombre_carpeta = "F:/Reportes SF/Vida Security/"
nombre_archivo_final = nombre_carpeta + str(dia) + "/" + nombre_reporte + ".xlsx"
Archivo_origen = nombre_carpeta + "Datos/" + nombre_reporte + ".xlsx"
crear_carpeta(nombre_carpeta)
actualizar(Archivo_origen)
Asunto = "Detalle de Uso " + nombre_reporte + " - " + str(dia)
shutil.copy2(Archivo_origen, nombre_archivo_final, follow_symlinks=True)

# create Excel application object
excel = win32.gencache.EnsureDispatch('Excel.Application')

# set ScreenUpdating property to False to hide Excel application
excel.ScreenUpdating = False

# open workbook
workbook = excel.Workbooks.Open(nombre_archivo_final)

# do some work with the workbook

# save and close workbook
workbook.Save()
workbook.Close()

# set ScreenUpdating property to True to show Excel application
excel.ScreenUpdating = True

# quit Excel application
excel.Quit()

# release COM object
del excel

envio_de_email(
    destinatarios=["marialaura.perez@security.cl",],
    concopia=[
        "dnunez@careassistance.com",
        "dgarcia@careassistance.com",
        "pcerda@careassistance.com",
        "frubio@careassistance.com",
    ],
    asunto=Asunto,
    nombre_reporte=str(nombre_reporte),
    ruta_adjunto=nombre_archivo_final,
    nombre_adjunto=nombre_reporte + ".xlsx",
    activo=1,
)

NameError: name 'dia' is not defined

In [ ]:
import win32com.client as win32
import os
import shutil
import datetime as dt

def crear_carpeta(nombre_carpeta):
    if not os.path.exists(nombre_carpeta):
        os.makedirs(nombre_carpeta)

def actualizar(Archivo_origen):
    excel = win32.gencache.EnsureDispatch('Excel.Application')
    wb = excel.Workbooks.Open(Archivo_origen)
    wb.RefreshAll()
    wb.Save()
    wb.Close()

def envio_de_email(destinatarios, concopia, asunto, nombre_reporte, ruta_adjunto, nombre_adjunto):
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = ";".join(destinatarios)
    mail.CC = ";".join(concopia)
    mail.Subject = asunto
    mail.Body = "Buen día,\n\nSe adjunta el detalle de uso de la aplicación " + nombre_reporte + " correspondiente al día de ayer.\n\nSaludos cordiales,\n\n"
    mail.Attachments.Add(ruta_adjunto, DisplayName=nombre_adjunto)
    mail.Send()

nombre_reporte = "Zurich - AMSA"
nombre_carpeta = "F:/Reportes SF/Zurich/"
nombre_archivo_final = nombre_carpeta + str(dt.date.today()) + "/" + nombre_reporte + ".xlsx"
Archivo_origen = nombre_carpeta + "Datos/" + nombre_reporte + ".xlsx"
crear_carpeta(nombre_carpeta)
actualizar(Archivo_origen)
Asunto = "Detalle de Uso " + nombre_reporte + " - " + str(dt.date.today())
shutil.copy2(Archivo_origen, nombre_archivo_final, follow_symlinks=True)

envio_de_email(
    destinatarios=[
        "clara.angelica.munoz@zurich.com",
        "claudia.ivonne.zapata@zurich.com",
        "maria.moreno.poblete@zurich.com",
        "nadia.pino@zurich.com",
    ],
    concopia=[
        "dgarcia@careassistance.com",
        "dnunez@careassistance.com",
        "pcerda@careassistance.com",
        "frubio@careassistance.com",
    ],
    asunto=Asunto,
    nombre_reporte=str(nombre_reporte),
    ruta_adjunto=nombre_archivo_final,
    nombre_adjunto=nombre_reporte + ".xlsx"